In [ ]:
!pip install pandas nltk bertopic youtube_comment_downloader

In [24]:
from itertools import islice
from youtube_comment_downloader import *

## data crawling
downloader = YoutubeCommentDownloader()
comments = downloader.get_comments_from_url('https://www.youtube.com/watch?v=LTduwK0-sGI', sort_by=SORT_BY_POPULAR)
for comment in islice(comments, 10):
    print(comment)
    
comments_list = list(comments)

output_file = "ENDEVR.json"
with open(output_file, "w", encoding="utf-8") as f:
    for comment in comments_list:
        json_line = json.dumps(comment, ensure_ascii=False)
        f.write(json_line + "\n")

print(f"Saved {len(comments_list)} comments to '{output_file}'")

✅ Saved 1569 comments to 'ENDEVR.json' in JSON Lines format


In [30]:
## preprocessing

import json, re, string
import pandas as pd
import nltk
from nltk.corpus import stopwords

input_data = "ENDEVR.json"
out_berttopic = "ENDEVR_btopic"
out_sent = "ENDEVR_sent"
out_emo = "ENDEVR_emo"
data = []
with open(input_data, "r") as f:
    for line in f:
        line = line.strip()
        if line:
            try:
                data.append(json.loads(line))
            except:
                pass

df = pd.DataFrame(data)
print(f"Loaded {len(df)} comments")
df = df.dropna(subset=["text"])
df["text"] = df["text"].astype(str)

def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"http\S+|www\S+", "", txt)
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    txt = re.sub(r"\s+", " ", txt)
    return txt.strip()

df["clean"] = df["text"].apply(clean_text)
print (df["clean"][5])

Loaded 1569 comments
a couple of years ago i had this dream where i was coming back home at night during winter i recall walking alone down the quiet streets while it was snowing as soon as i arrived home i saw my mother cooking some dinner however i was older which means that she wasnt alive at that time it was then that i realized she was a robot it had some cables at the back of its head overwhelmed with emotion i started to cry as i hugged the robot remembering that my real mother had died it was an eerie dream and im convinced its not too far from reality as robots will never truly replace humans in emotional connections


In [32]:
## analysis: topic modeling with BERTopic

import matplotlib.pyplot as plt
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

stop_words = stopwords.words("english")

vectorizer = CountVectorizer(stop_words=stop_words)
topic_model = BERTopic(vectorizer_model=vectorizer,
                       language="english",
                       verbose=True,
                       calculate_probabilities=False)
topics, probs = topic_model.fit_transform(df["clean"])

## without stopwords removal
# topic_model = BERTopic(language="english", verbose=True, calculate_probabilities=False)
# topics, probs = topic_model.fit_transform(df["clean"])

df["topic"] = topics
topic_info = topic_model.get_topic_info()

print("\nTop 10 Topics:\n")
print(topic_info.head(10))

df.to_csv(out_berttopic+"_2.csv", index=False)
print("Done! BERTopic results saved")

for i in topic_info.head(11)["Topic"]:
    if i == -1:  # -1 means outliers
        continue
    print(f"\n--- Topic {i}: {topic_model.get_topic(i)} ---")
    sample_comments = df[df["topic"] == i]["text"].head(6).tolist()
    for c in sample_comments:
        print("•", c[:500], "...\n")

2025-11-11 15:15:19,995 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|█████████████████████| 50/50 [00:43<00:00,  1.16it/s]
2025-11-11 15:16:05,754 - BERTopic - Embedding - Completed ✓
2025-11-11 15:16:05,755 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-11 15:16:08,480 - BERTopic - Dimensionality - Completed ✓
2025-11-11 15:16:08,484 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-11 15:16:08,528 - BERTopic - Cluster - Completed ✓
2025-11-11 15:16:08,530 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-11 15:16:08,573 - BERTopic - Representation - Completed ✓



Top 10 Topics:

   Topic  Count                                  Name  \
0     -1    510                -1_world_god_life_dont   
1      0    446                  0_ai_soul_would_body   
2      1    113            1_forever_live_want_living   
3      2     66       2_deepak_scooby7877_aq2330_blah   
4      3     54  3_documentary_movie_terminator_video   
5      4     50           4_jesus_christ_life_eternal   
6      5     47     5_immortality_immortal_death_life   
7      6     43               6_god_choiche_day_every   
8      7     34                 7_shall_death_seek_96   
9      8     31         8_chucky_double_antonia_edged   

                                      Representation  \
0  [world, god, life, dont, im, want, live, like,...   
1  [ai, soul, would, body, consciousness, human, ...   
2  [forever, live, want, living, dont, wants, wou...   
3  [deepak, scooby7877, aq2330, blah, im, 100, ni...   
4  [documentary, movie, terminator, video, episod...   
5  [jesus, christ, 

In [33]:
# !pip install transformers torch --quiet
from transformers import pipeline

model_emo_rlarge = "cardiffnlp/twitter-roberta-large-emotion-latest"
model_emo_rbase = "cardiffnlp/twitter-roberta-base-emotion-latest"
model_sent_rbase = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model_emo_dbertbase ="bhadresh-savani/distilbert-base-uncased-emotion"

# Load sentiment analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model=model_emo_rlarge
)

Device set to use cpu


In [35]:
## analysis: sentiment/emotion analysis via Transformer-based LLM

## emotion labels
id2label = { 
    "0": "anger", "1": "anticipation", "2": "disgust", "3": "fear",
    "4": "joy", "5": "love", "6": "optimism", "7": "pessimism",
    "8": "sadness", "9": "surprise", "10": "trust" 
}

## sentiment labels
id2label_sent = { 
    "0": "Negative", "1": "Neutral", "2": "Positive"
}

## using emotion model
def get_emotion_hf(text):
    try:
        result = sentiment_model(text[:256])[0]  # truncate to 256 tokens
        label_id = result["label"].replace("LABEL_", "")  # extract numeric ID
        label_name = id2label.get(label_id, result["label"])  # map to emotion
        return label_name, float(result["score"])
    except:
        return None, None

## using sentiment model
def get_sentiment_hf(text):
    try:
        result = sentiment_model(text[:256])[0]   # truncate to 512 tokens
        label_id = result["label"].replace("LABEL_", "")  # extract numeric ID
        label_name = id2label_sent.get(label_id, result["label"])  # map to emotion
        return label_name, float(result["score"])
        # return result["label"], float(result["score"])
    except:
        return None, None

## sentiment analysis
df[["hf_label", "hf_score"]] = df["clean"].apply(lambda x: pd.Series(get_sentiment_hf(x)))
print(df["hf_label"].value_counts())
df.to_csv(out_sent+".csv", index=False)
print(f"✅ Predictions saved as {out_sent}.csv")

## emotion analysis
df[["hf_label", "hf_score"]] = df["clean"].apply(lambda x: pd.Series(get_emotion_hf(x)))
print(df["hf_label"].value_counts())
df.to_csv(out_emo+".csv", index=False)
print(f"✅ Predictions saved as {out_sent}.csv")

hf_label
optimism        369
anticipation    287
sadness         254
joy             233
disgust         206
anger            99
fear             75
surprise         23
love             14
pessimism         8
Name: count, dtype: int64
✅ Predictions saved as ENDEVR_sent.csv
